In [1]:
import tensorflow as tf
import numpy as np
np.random.seed(0)
np.set_printoptions(2)

In [2]:
n = 8
actual_y_indices = np.random.randint(0, 3, (n,))
print(actual_y_indices)
y = np.array([2, 4, 5])[actual_y_indices]
print(y)
classes = np.unique(y)
print(classes)
y = y[:, np.newaxis]

[0 1 0 1 1 2 0 2]
[2 4 2 4 4 5 2 5]
[2 4 5]


In [3]:
y_one_hot = y == classes
print(y_one_hot)

[[ True False False]
 [False  True False]
 [ True False False]
 [False  True False]
 [False  True False]
 [False False  True]
 [ True False False]
 [False False  True]]


In [4]:
y_indices = np.where(y_one_hot)[1]
print(y_indices)

[0 1 0 1 1 2 0 2]


In [5]:
p_query = np.random.rand(n, 3)
p_query = (p_query.T / np.sum(p_query.T, axis=0)).T
print(p_query)

[[ 0.07  0.34  0.59]
 [ 0.48  0.28  0.23]
 [ 0.46  0.19  0.36]
 [ 0.25  0.65  0.1 ]
 [ 0.41  0.22  0.37]
 [ 0.27  0.35  0.38]
 [ 0.31  0.29  0.4 ]
 [ 0.14  0.62  0.24]]


In [6]:
p_field = p_query[:, y_indices]
print(p_field)

[[ 0.07  0.34  0.07  0.34  0.34  0.59  0.07  0.59]
 [ 0.48  0.28  0.48  0.28  0.28  0.23  0.48  0.23]
 [ 0.46  0.19  0.46  0.19  0.19  0.36  0.46  0.36]
 [ 0.25  0.65  0.25  0.65  0.65  0.1   0.25  0.1 ]
 [ 0.41  0.22  0.41  0.22  0.22  0.37  0.41  0.37]
 [ 0.27  0.35  0.27  0.35  0.35  0.38  0.27  0.38]
 [ 0.31  0.29  0.31  0.29  0.29  0.4   0.31  0.4 ]
 [ 0.14  0.62  0.14  0.62  0.62  0.24  0.14  0.24]]


Expanding matrices

In [7]:
tf_y_indices = tf.placeholder(tf.int32, shape=list(y_indices.shape))
tf_p_query = tf.placeholder(tf.float32, shape=list(p_query.shape))

In [8]:
tf_p_field = tf.transpose(tf.gather(tf.transpose(tf_p_query), tf_y_indices))
print(tf_p_field)

Tensor("transpose_1:0", shape=(8, 8), dtype=float32)


In [9]:
sess = tf.Session()
print(sess.run(tf_p_field, feed_dict={tf_y_indices: y_indices, tf_p_field: p_field}))

[[ 0.07  0.34  0.07  0.34  0.34  0.59  0.07  0.59]
 [ 0.48  0.28  0.48  0.28  0.28  0.23  0.48  0.23]
 [ 0.46  0.19  0.46  0.19  0.19  0.36  0.46  0.36]
 [ 0.25  0.65  0.25  0.65  0.65  0.1   0.25  0.1 ]
 [ 0.41  0.22  0.41  0.22  0.22  0.37  0.41  0.37]
 [ 0.27  0.35  0.27  0.35  0.35  0.38  0.27  0.38]
 [ 0.31  0.29  0.31  0.29  0.29  0.4   0.31  0.4 ]
 [ 0.14  0.62  0.14  0.62  0.62  0.24  0.14  0.24]]


Finding indices

In [10]:
tf_y = tf.placeholder(tf.float32, shape=list(y.shape))
tf_classes = tf.cast(tf.constant(classes), tf.float32)
tf_y_one_hot = tf.equal(tf_y, tf_classes)
tf_y_indices = tf.where(tf_y_one_hot)[:, 1]
print(tf_y_indices)

Tensor("strided_slice:0", shape=(?,), dtype=int64)


In [11]:
sess = tf.Session()
print(sess.run(tf_y_indices, feed_dict={tf_y: y}))

[0 1 0 1 1 2 0 2]


In [19]:
from sklearn.metrics import log_loss
ll = -np.sum(np.log(p_query) * y_one_hot)/n
print(ll)
ll = -np.sum(np.log(p_query[np.arange(y_indices.shape[0]), y_indices])) / n
print(ll)
print(log_loss(y_indices, p_query))

1.27390177768
1.27390177768
1.27390177768


In [27]:
# classes = np.unique(y)
# class_indices = np.arange(classes.shape[0])
# tf_classes = tf.cast(tf.constant(classes), tf.float32)
# tf_class_indices = tf.cast(tf.constant(class_indices), tf.int32)

ind = tf.stack([tf.constant(np.arange(n)), y_indices], axis=1)
tf_p_want = tf.gather_nd(p_query, ind)
tf_ll = -tf.reduce_sum(tf.log(tf_p_want)) / n
print(sess.run(tf_ll, feed_dict={tf_y: y}))

1.27390177768
